In [2]:
import sys, os, glob
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoModel, AutoTokenizer
from transformers.models.bert.configuration_bert import BertConfig
from DGXutils import GetFileNames, GetLowestGPU

In [4]:
device = GetLowestGPU()
model_path = 'zhangtaolab/dnabert2-conservation'

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(f"{model_path}", trust_remote_code=True)

config = BertConfig.from_pretrained(f"{model_path}")
model = AutoModel.from_pretrained(f"{model_path}", trust_remote_code=True, config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

bert_layers.py:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/zhangtaolab/dnabert2-conservation:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


bert_padding.py:   0%|          | 0.00/6.10k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhangtaolab/dnabert2-conservation:
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/zhangtaolab/dnabert2-conservation:
- bert_layers.py
- flash_attn_triton.py
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/468M [00:00<?, ?B/s]

/home/pkr/.cache/huggingface/modules/transformers_modules/zhangtaolab/dnabert2-conservation/5babb4ae2a106150ab00cb4487d95f83293b9dc6/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


In [5]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors="pt").to(device)
print(inputs['input_ids'])

tensor([[   1,    5,  194,   32,  757, 1239, 2092,  294,   24,  359,   88,   93,
           32,   75,   77,   19,    2]], device='cuda:0')


In [11]:
seq, cls = model(**inputs)

In [27]:
model.config.hidden_size

768

In [31]:
list = [1, 2, 3]
list2 = [*[1, 2, 3 for _ in range(2)]]
print(list2)

SyntaxError: invalid syntax (1086831590.py, line 2)